In [454]:
%reload_ext autoreload
%autoreload 2
import data_01_conflictModel as model
import data_02_conflictSolvers as solvers
#import data_04_spSolvers as solvers
import data_03_gmcrUtilities as gmcrUtils
import gmcr
import numpy as np
from utilities import *

In [455]:
myConflict = gmcr.model.ConflictModel()

In [456]:
payoffs = load_data("../model/data/processed/50yr_samples.pickle")

In [457]:
baseline = {'hydropower_revenue':39.30, 'atomic_power_plant_discharge':0.633,
       'baltimore_discharge':1.00, 'chester_discharge':1.00, 'recreation':0.00, 'environment':0.1058,
       'flood_duration':1.633}

In [458]:
payoffs

,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment,flood_duration
best_app,33.428360,0.675815,0.450959,0.602542,0.750714,-0.116995,-1.666667
best_ch,44.640454,0.667632,0.581992,0.578373,0.768214,-0.045616,-1.000000
best_comp,47.521088,0.675057,0.408810,0.581773,0.839643,-0.032985,-0.833333
best_dfl,36.452284,0.025641,0.277056,0.012075,-0.000000,-0.000000,-0.000000
best_env,41.245030,0.669965,0.499875,0.603974,0.768214,-0.086025,-1.666667
best_hydro,47.752741,0.675139,0.053342,0.596784,0.821429,-0.048001,-0.833333
best_rec,42.795952,0.675009,0.577004,0.604840,0.750714,-0.083129,-1.241667


In [459]:
payoffs = payoffs[payoffs.index.isin(['best_app', 'best_ch', 'best_comp'])]

In [460]:
payoffs = payoffs.append(baseline, ignore_index = True)

In [431]:
#payoffs = payoffs[['atomic_power_plant_discharge', 'hydropower_revenue']]

In [461]:
ranks = payoffs.rank(ascending = False)

In [462]:
hydropower_revenue = gmcr.model.DecisionMaker(myConflict, "hydropower_revenue")
myConflict.decisionMakers.append(hydropower_revenue)
atomic_power_plant_discharge = gmcr.model.DecisionMaker(myConflict, "atomic_power_plant_discharge")
myConflict.decisionMakers.append(atomic_power_plant_discharge)
baltimore_discharge = gmcr.model.DecisionMaker(myConflict, "baltimore_discharge")
myConflict.decisionMakers.append(baltimore_discharge)
chester_discharge = gmcr.model.DecisionMaker(myConflict, "chester_discharge")
myConflict.decisionMakers.append(chester_discharge)
recreation = gmcr.model.DecisionMaker(myConflict, "recreation")
myConflict.decisionMakers.append(recreation)
environment = gmcr.model.DecisionMaker(myConflict, "environment")
myConflict.decisionMakers.append(environment)
flood_duration = gmcr.model.DecisionMaker(myConflict, "flood_duration")
myConflict.decisionMakers.append(flood_duration)
print(myConflict.decisionMakers)

['hydropower_revenue', 'atomic_power_plant_discharge', 'baltimore_discharge', 'chester_discharge', 'recreation', 'environment', 'flood_duration']


In [463]:
myConflict.useManualPreferenceRanking = True

In [464]:
nr_decisions = payoffs.shape[0]-1
nr_players = payoffs.shape[1]
for i in range(0, nr_players):
    for j in range(0,nr_decisions):
        myConflict.decisionMakers[i].addOption(str(j))

In [465]:
%%time
myConflict.recalculateFeasibleStates()

7 players
3 decisions
Wall time: 124 ms


In [466]:
temp = [(x,y) for x,y in zip([feas for feas in myConflict.feasibles.name], [feas for feas in myConflict.feasibles])]

In [467]:
dict = {}
for i in set([feas for feas in myConflict.feasibles.name]):
    dict[i] = []
for i in temp:
    dict[i[0]].append(i[1])

In [468]:
ranks = payoffs.rank(ascending = False)
ranks = ranks.reset_index()
ranks['index']= ranks['index'].replace(nr_decisions, 'baseline')

In [469]:
actors = [dm.name for dm in myConflict.decisionMakers]

In [442]:
ranks

,index,atomic_power_plant_discharge,hydropower_revenue
0,0,1.0,3.0
1,1,2.0,1.0
2,baseline,3.0,2.0


In [470]:
for i in range((nr_players)):
    #rank_order = (np.array(list(ranks.iloc[:,i]))-1).astype(int)
    #rank_order = np.where(rank_order == np.max(rank_order), 'baseline', rank_order)
    rank_order = list(ranks.sort_values(by = actors[i])['index'])
    #print(rank_order)
    index_map = {v:j for j, v in enumerate(rank_order)}
    sorted_list = sorted(dict.items(), key=lambda x: index_map[x[0]])
    myConflict.decisionMakers[i].preferenceRanking = list(list(zip(*sorted_list))[1])

In [471]:
for i in range((nr_players)):
     myConflict.decisionMakers[i].calculatePreferences()

In [445]:
myConflict.feasibles.name

[0, 'baseline', 'baseline', 1]

In [447]:
hydropower_revenue.preferenceRanking

[[3], [1, 2], [0]]

In [472]:
%%time
lSolver = gmcr.solvers.LogicalSolver(myConflict)

Wall time: 7min 3s


In [473]:
%%time
all = lSolver.findEquilibria()

Wall time: 11.4 s


In [474]:
decisions = np.array(myConflict.feasibles.name)

In [475]:
set(decisions[lSolver.gmrEquilibria])

{'baseline'}

In [476]:
payoffs

,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment,flood_duration
0,33.428360,0.675815,0.450959,0.602542,0.750714,-0.116995,-1.666667
1,44.640454,0.667632,0.581992,0.578373,0.768214,-0.045616,-1.000000
2,47.521088,0.675057,0.408810,0.581773,0.839643,-0.032985,-0.833333
3,39.300000,0.633000,1.000000,1.000000,0.000000,0.105800,1.633000


In [477]:
print(f'Nash Equilibrium: {set(decisions[lSolver.nashEquilibria])}',
f'GMR Equilirbium: {set(decisions[lSolver.gmrEquilibria])}',
f'SEQ Equilibrium: {set(decisions[lSolver.seqEquilibria])}',
f'SIM Equilibrium: {set(decisions[lSolver.simEquilibria])}',
f'SEQ SIM Equilibrium: {set(decisions[lSolver.seqSimEquilibria])}',
f'SMR Equilirbium: {set(decisions[lSolver.smrEquilibria])}')

Nash Equilibrium: {'baseline'} GMR Equilirbium: {'baseline'} SEQ Equilibrium: {'baseline'} SIM Equilibrium: {'baseline'} SEQ SIM Equilibrium: {'baseline'} SMR Equilirbium: {'baseline'}
